In [1]:
## Load the data

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('./data.csv')

In [4]:
## Clean the data

In [5]:
data.columns

Index(['date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'street', 'city',
       'statezip', 'country'],
      dtype='object')

In [6]:
data.drop(['sqft_living','sqft_lot','waterfront','view','condition','sqft_above','sqft_basement','street','city','statezip','country'],axis=1,inplace=True)

In [7]:
data.drop('date',axis=1,inplace=True)

In [8]:
data.head()

       price  bedrooms  bathrooms  floors  yr_built  yr_renovated
0   313000.0       3.0       1.50     1.5      1955          2005
1  2384000.0       5.0       2.50     2.0      1921             0
2   342000.0       3.0       2.00     1.0      1966             0
3   420000.0       3.0       2.25     1.0      1963             0
4   550000.0       4.0       2.50     1.0      1976          1992

In [9]:
## Feature Enginnering

In [10]:
def fe(data,col):
    print(len(data))
    max_no = data[col].quantile(0.99)
    min_no = data[col].quantile(0.05)
    data = data[data[col] > min_no]
    data = data[data[col] < max_no]
    print(len(data))
    return data

In [11]:
for col in list(data.columns):
    print(col)
    data = fe(data,'price')

In [12]:
data.head()

      price  bedrooms  bathrooms  floors  yr_built  yr_renovated
2  342000.0       3.0       2.00     1.0      1966             0
3  420000.0       3.0       2.25     1.0      1963             0
4  550000.0       4.0       2.50     1.0      1976          1992
5  490000.0       2.0       1.00     1.0      1938          1994
6  335000.0       2.0       2.00     1.0      1976             0

In [13]:
X = data.drop('price',axis=1)
y = data['price']

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

In [16]:
len(X_train),len(X_test)

(2367, 789)

In [17]:
## Modelling

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim

In [19]:
import torch.nn.functional as F

In [20]:
class BaseLine_Model(nn.Module):
    def __init__(self,input_shape,output_shape):
        super().__init__()
        self.fc1 = nn.Linear(input_shape,64)
        self.fc2 = nn.Linear(64,128)
        self.fc3 = nn.Linear(128,256)
        self.fc4 = nn.Linear(256,128)
        self.fc5 = nn.Linear(128,output_shape)
    
    def forward(self,X):
        preds = self.fc1(X)
        preds = F.relu(preds)
        preds = self.fc2(preds)
        preds = F.relu(preds)
        preds = self.fc3(preds)
        preds = F.relu(preds)
        preds = self.fc4(preds)
        preds = F.relu(preds)
        preds = self.fc5(preds)
        return preds

In [21]:
EPOCHS = 250

In [22]:
import wandb

In [23]:
BATCH_SIZE = 32

In [24]:
PROJECT_NAME = 'House-Price-Pred'

In [25]:
from tqdm import tqdm

In [26]:
device = torch.device('cuda')

In [27]:
model = BaseLine_Model(5,1).to(device)

In [28]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr=0.1)

In [29]:
def get_loss(criterion,X,y,model):
    preds = model(X.float().to(device))
    preds = preds.to(device)
    y = y.to(device)
#     criterion.to(device)
    loss = criterion(preds,y)
    return loss.item()
def get_accuracy(X,y,model):
    correct = 0
    total = 0
    for i in range(len(X)):
        pred = model(X[i].float().to(device))
        pred.to(device)
        if round(pred[0]) == round(y[i]):
            correct += 1
        total += 1
    if correct == 0:
        correct += 1
    return round(correct/total,3)

In [30]:
import numpy as np

In [31]:
X_train = torch.from_numpy(np.array(X_train))
y_train = torch.from_numpy(np.array(y_train))
X_test = torch.from_numpy(np.array(X_test))
y_test = torch.from_numpy(np.array(y_test))

In [32]:
get_accuracy(X_test,y_test,model)

In [33]:
def get_loss(criterion,X,y,model):
    preds = model(X.float().to(device))
    preds = preds.to(device)
    y = y.to(device)
#     criterion.to(device)
    loss = criterion(preds,y)
    return loss.item()
def get_accuracy(X,y,model):
    correct = 0
    total = 0
    for i in range(len(X)):
        pred = model(X[i].float().to(device))
        pred.to(device)
        if round(int(pred[0])) == round(int(y[i])):
            correct += 1
        total += 1
    if correct == 0:
        correct += 1
    return round(correct/total,3)

In [34]:
import numpy as np

In [35]:
X_train = torch.from_numpy(np.array(X_train))
y_train = torch.from_numpy(np.array(y_train))
X_test = torch.from_numpy(np.array(X_test))
y_test = torch.from_numpy(np.array(y_test))

In [36]:
get_accuracy(X_test,y_test,model)

0.001

In [37]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    for i in range(0,len(X_train),BATCH_SIZE):
        X_batch = X_train[i:i+BATCH_SIZE].to(device)
        y_batch = y_train[i:i+BATCH_SIZE].to(device)
        model.to(device)
        preds = model(X_batch.float())
        preds = preds.view(len(preds))
        preds.to(device)
        loss = criterion(preds.float(),y_batch.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})

In [38]:
preds

tensor([480747.5938, 563435.1250, 610842.6250, 420845.0938, 463107.5938,
        486627.5938, 604227.6250, 632157.6250, 523745.0938, 472662.5938,
        560495.1250, 552410.1250, 467517.5938, 693162.6250, 572622.6250,
        592467.6250, 580707.6250, 469355.0938, 596877.6250, 648695.1250,
        552410.1250, 525582.6250, 572990.1250, 500592.5938, 552410.1250,
        590997.6250, 613415.1250, 521172.5938, 683975.1250, 624807.6250,
        549102.6250], device='cuda:0', grad_fn=<ViewBackward>)

In [39]:
y_batch

tensor([ 359000.,  535000.,  750000.,  435000.,  552000.,  471001.,  340000.,
         744000.,  550000.,  446000.,  947500.,  715000.,  339000., 1125000.,
         650000.,  450000.,  344000.,  360000.,  440000., 1149000.,  405500.,
         849900.,  680000.,  380000.,  450000.,  347500.,  402500.,  818000.,
         690000.,  955000.,  330000.], device='cuda:0', dtype=torch.float64)

In [40]:
torch.round(preds)

tensor([480748., 563435., 610843., 420845., 463108., 486628., 604228., 632158.,
        523745., 472663., 560495., 552410., 467518., 693163., 572623., 592468.,
        580708., 469355., 596878., 648695., 552410., 525583., 572990., 500593.,
        552410., 590998., 613415., 521173., 683975., 624808., 549103.],
       device='cuda:0', grad_fn=<RoundBackward>)

In [41]:
torch.round(y_batch)

tensor([ 359000.,  535000.,  750000.,  435000.,  552000.,  471001.,  340000.,
         744000.,  550000.,  446000.,  947500.,  715000.,  339000., 1125000.,
         650000.,  450000.,  344000.,  360000.,  440000., 1149000.,  405500.,
         849900.,  680000.,  380000.,  450000.,  347500.,  402500.,  818000.,
         690000.,  955000.,  330000.], device='cuda:0', dtype=torch.float64)

In [42]:
EPOCHS = 50

In [43]:
EPOCHS = 25

In [44]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    for i in range(0,len(X_train),BATCH_SIZE):
        X_batch = X_train[i:i+BATCH_SIZE].to(device)
        y_batch = y_train[i:i+BATCH_SIZE].to(device)
        model.to(device)
        preds = model(X_batch.float())
        preds = preds.view(len(preds))
        preds.to(device)
        loss = criterion(preds.float(),y_batch.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})